## 1. Feature engineering

In this notebook, we clean the raw data then extract new feature, the target of our prediction.
The code will be converted into a script for Sagemaker processing jobs in the next notebook.
Therefore, we build code onto one example raw data file here.

In [3]:
# install a package to read parquet files
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 12.6 MB/s eta 0:00:0000:0100:01


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
columns_to_read = [
    "tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_distance", "PULocationID", "total_amount"
]
parquet_files = ["./taxi/yellow_tripdata_2019-02.parquet"]

df_raw = pd.DataFrame()
for f in parquet_files:
    t = pd.read_parquet(f, columns=columns_to_read)
    df_raw = pd.concat([df_raw, t])

In [6]:
df = df_raw

In [7]:
df

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,total_amount
0,2019-02-01 00:59:04,2019-02-01 01:07:27,2.10,48,12.3
1,2019-02-01 00:33:09,2019-02-01 01:03:58,9.80,230,33.3
2,2019-02-01 00:09:03,2019-02-01 00:09:16,0.00,145,3.8
3,2019-02-01 00:45:38,2019-02-01 00:51:10,0.80,95,6.8
4,2019-02-01 00:25:30,2019-02-01 00:28:14,0.80,140,6.3
...,...,...,...,...,...
7049365,2019-02-28 23:00:00,2019-02-28 23:15:00,7.64,9,39.5
7049366,2019-02-28 23:27:00,2019-03-01 00:04:00,7.47,100,43.0
7049367,2019-02-28 23:42:10,2019-03-01 00:22:12,14.42,161,60.0
7049368,2019-02-28 23:20:26,2019-02-28 23:37:14,2.53,181,36.0


In [8]:
print(df.isnull().sum())

tpep_pickup_datetime     0
tpep_dropoff_datetime    0
trip_distance            0
PULocationID             0
total_amount             0
dtype: int64


In [9]:
df = df.dropna()

In [10]:
df.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
trip_distance                   float64
PULocationID                      int64
total_amount                    float64
dtype: object

In [11]:
'''
This code is to validate the date time format. However, we don't need this as the format is valid for all rows.

!pip install datatest
from datetime import datetime
from datatest import validate


def strftime_format(format):
    def func(value):
        try:
            datetime.strptime(value, format)
        except ValueError:
            return False
        return True
    func.__doc__ = f'should use date format {format}'
    return func


#data = ['2020-02-29', '03-17-2021', '2021-02-29', '2021-04-01']
#validate(data, strftime_format('%Y-%m-%d'))
#df['tpep_dropoff_datetime']

validate(df['tpep_pickup_datetime'].tolist(), strftime_format('%Y-%m-%d %H:%M:%S'))
validate(df['tpep_dropoff_datetime'].tolist(), strftime_format('%Y-%m-%d %H:%M:%S'))
'''

"\nThis code is to validate the date time format. However, we don't need this as the format is valid for all rows.\n\n!pip install datatest\nfrom datetime import datetime\nfrom datatest import validate\n\n\ndef strftime_format(format):\n    def func(value):\n        try:\n            datetime.strptime(value, format)\n        except ValueError:\n            return False\n        return True\n    func.__doc__ = f'should use date format {format}'\n    return func\n\n\n#data = ['2020-02-29', '03-17-2021', '2021-02-29', '2021-04-01']\n#validate(data, strftime_format('%Y-%m-%d'))\n#df['tpep_dropoff_datetime']\n\nvalidate(df['tpep_pickup_datetime'].tolist(), strftime_format('%Y-%m-%d %H:%M:%S'))\nvalidate(df['tpep_dropoff_datetime'].tolist(), strftime_format('%Y-%m-%d %H:%M:%S'))\n"

In [12]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [13]:
df

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,total_amount,duration
0,2019-02-01 00:59:04,2019-02-01 01:07:27,2.10,48,12.3,8.383333
1,2019-02-01 00:33:09,2019-02-01 01:03:58,9.80,230,33.3,30.816667
2,2019-02-01 00:09:03,2019-02-01 00:09:16,0.00,145,3.8,0.216667
3,2019-02-01 00:45:38,2019-02-01 00:51:10,0.80,95,6.8,5.533333
4,2019-02-01 00:25:30,2019-02-01 00:28:14,0.80,140,6.3,2.733333
...,...,...,...,...,...,...
7049365,2019-02-28 23:00:00,2019-02-28 23:15:00,7.64,9,39.5,15.000000
7049366,2019-02-28 23:27:00,2019-03-01 00:04:00,7.47,100,43.0,37.000000
7049367,2019-02-28 23:42:10,2019-03-01 00:22:12,14.42,161,60.0,40.033333
7049368,2019-02-28 23:20:26,2019-02-28 23:37:14,2.53,181,36.0,16.800000


In [14]:
'''
It turned out that rolling does not help much with prediction.

# each row of this series is actually independent
# no mean to impute nulls from previous values
# just for a practice, we impute using a mean of last 10 

df['mean_last_10'] = df['duration'].rolling(window=10, min_periods=1).mean()
df
'''

"\nIt turned out that rolling does not help much with prediction.\n\n# each row of this series is actually independent\n# no mean to impute nulls from previous values\n# just for a practice, we impute using a mean of last 10 \n\ndf['mean_last_10'] = df['duration'].rolling(window=10, min_periods=1).mean()\ndf\n"

In [15]:
year = "2019"
df = df[df['tpep_pickup_datetime'] >= f"{year}-01-01 00:00:00"]
df = df[df['tpep_pickup_datetime'] <= f"{year}-12-31 11:59:59"]

In [16]:
df = df[df['trip_distance'] >= 0]
df = df[df['total_amount'] >= 0]
df = df[df['duration'] >= 0]

In [17]:
df.shape

(7039804, 6)

In [18]:
df.shape

(7039804, 6)

In [19]:
df.describe()

,trip_distance,PULocationID,total_amount,duration
count,7.039804e+06,7.039804e+06,7.039804e+06,7.039804e+06
mean,2.920032e+00,1.634032e+02,1.873045e+01,1.729845e+01
std,3.820722e+00,6.599521e+01,2.556780e+02,7.119803e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,9.500000e-01,1.140000e+02,1.115000e+01,6.466667e+00
50%,1.600000e+00,1.620000e+02,1.430000e+01,1.083333e+01
75%,2.960000e+00,2.330000e+02,2.000000e+01,1.773333e+01
max,7.015000e+02,2.650000e+02,6.711249e+05,2.598798e+04


In [20]:
df['trip_distance'].value_counts(bins=10)

(-0.7030000000000001, 70.15]    7039737
(70.15, 140.3]                       63
(140.3, 210.45000000000002]           2
(210.45000000000002, 280.6]           1
(631.35, 701.5]                       1
(280.6, 350.75]                       0
(350.75, 420.90000000000003]          0
(420.90000000000003, 491.05]          0
(491.05, 561.2]                       0
(561.2, 631.35]                       0
Name: trip_distance, dtype: int64

In [21]:
df['total_amount'].value_counts(bins=10)

(-671.126, 67112.494]                       7039802
(67112.494, 134224.988]                           1
(604012.446, 671124.9400000001]                   1
(134224.988, 201337.48200000002]                  0
(201337.48200000002, 268449.976]                  0
(268449.976, 335562.47000000003]                  0
(335562.47000000003, 402674.96400000004]          0
(402674.96400000004, 469787.458]                  0
(469787.458, 536899.952]                          0
(536899.952, 604012.446]                          0
Name: total_amount, dtype: int64

In [22]:
df['duration'].value_counts(bins=10)

(-25.989, 2598.7980000000002]      7039801
(2598.7980000000002, 5197.597]           2
(23389.185, 25987.983]                   1
(5197.597, 7796.395]                     0
(7796.395, 10395.193000000001]           0
(10395.193000000001, 12993.992]          0
(12993.992, 15592.79]                    0
(15592.79, 18191.588]                    0
(18191.588, 20790.387]                   0
(20790.387, 23389.185]                   0
Name: duration, dtype: int64

In [23]:
outlier_columns = ['trip_distance', 'total_amount', 'duration']

sigma2 = 0.9544
sigma3 = 0.9973
sigma4 = 0.9999
for c in outlier_columns:
    s1 = df[c].quantile(sigma4)
    s0 = df[c].quantile(1-sigma4)
    print(f'{c} {s0} - {s1}')

    df = df.drop(df[df[c] > s1].index)
    df = df.drop(df[df[c] < s0].index)

trip_distance 0.0 - 40.500984999956565
total_amount 0.0 - 195.36180199999365
duration 0.0 - 1438.5166666666667


In [114]:
df['pickup_time'] = df['tpep_pickup_datetime'].dt.floor('h')
df = df.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [115]:
df

,trip_distance,PULocationID,total_amount,duration,pickup_time
0,2.10,48,12.3,8.383333,2019-02-01 00:00:00
1,9.80,230,33.3,30.816667,2019-02-01 00:00:00
2,0.00,145,3.8,0.216667,2019-02-01 00:00:00
3,0.80,95,6.8,5.533333,2019-02-01 00:00:00
4,0.80,140,6.3,2.733333,2019-02-01 00:00:00
...,...,...,...,...,...
7049365,7.64,9,39.5,15.000000,2019-02-28 23:00:00
7049366,7.47,100,43.0,37.000000,2019-02-28 23:00:00
7049367,14.42,161,60.0,40.033333,2019-02-28 23:00:00
7049368,2.53,181,36.0,16.800000,2019-02-28 23:00:00


In [116]:
df['count'] = df.groupby(by=['pickup_time', 'PULocationID'])['duration'].transform('size')
df

,trip_distance,PULocationID,total_amount,duration,pickup_time,count
0,2.10,48,12.3,8.383333,2019-02-01 00:00:00,317
1,9.80,230,33.3,30.816667,2019-02-01 00:00:00,379
2,0.00,145,3.8,0.216667,2019-02-01 00:00:00,15
3,0.80,95,6.8,5.533333,2019-02-01 00:00:00,1
4,0.80,140,6.3,2.733333,2019-02-01 00:00:00,53
...,...,...,...,...,...,...
7049365,7.64,9,39.5,15.000000,2019-02-28 23:00:00,3
7049366,7.47,100,43.0,37.000000,2019-02-28 23:00:00,312
7049367,14.42,161,60.0,40.033333,2019-02-28 23:00:00,598
7049368,2.53,181,36.0,16.800000,2019-02-28 23:00:00,13


In [118]:
df_agg = df.sort_values(by=['pickup_time', 'PULocationID'], ascending=True)\
    .groupby(by=['pickup_time', 'PULocationID'])\
    .mean(numeric_only=False)

In [119]:
df_agg

trip_distance  total_amount     duration  \
pickup_time         PULocationID                                             
2008-12-31 06:00:00 132                   19.61         65.00    28.000000   
2008-12-31 22:00:00 132                   15.35         52.56   112.850000   
2008-12-31 23:00:00 48                     4.62         21.30    20.350000   
                    50                     1.88         13.30    14.900000   
                    88                     3.69         16.30    14.833333   
...                                         ...           ...          ...   
2019-09-29 13:00:00 132                    4.86         22.25    14.750000   
2038-02-17 20:00:00 262                    0.56          9.36   202.033333   
                    263                    0.45          7.80     2.966667   
2038-02-17 21:00:00 113                    5.07         21.80  1397.450000   
                    249                    0.86          9.30     6.850000   

                                  count  
pickup_time         PULocationID         
2008-12-31 06:00:00 132             1.0  
2008-12-31 22:00:00 132             1.0  
2008-12-31 23:00:00 48              1.0  
                    50              1.0  
                    88              1.0  
...                                 ...  
2019-09-29 13:00:00 132             1.0  
2038-02-17 20:00:00 262             1.0  
                    263             1.0  
2038-02-17 21:00:00 113             1.0  
                    249             1.0  

[95224 rows x 4 columns]

In [53]:
df_agg = df_agg.reset_index()
df_agg

,pickup_time,PULocationID,trip_distance,total_amount,duration,count
0,2019-01-31 22:00:00,7,0.760000,5.800000,2.800000,1.0
1,2019-01-31 23:00:00,4,4.660000,20.160000,15.450000,1.0
2,2019-01-31 23:00:00,7,0.650000,5.300000,2.700000,1.0
3,2019-02-01 00:00:00,4,1.904444,11.807778,9.864815,18.0
4,2019-02-01 00:00:00,7,1.781667,9.592500,7.538889,12.0
...,...,...,...,...,...,...
2601,2019-02-28 22:00:00,10,11.790000,36.910000,21.250000,2.0
2602,2019-02-28 23:00:00,4,2.674091,17.392500,13.003030,44.0
2603,2019-02-28 23:00:00,7,2.104444,11.817778,9.396296,9.0
2604,2019-02-28 23:00:00,9,5.710000,23.960000,13.016667,3.0


**Below code is for exploratory analysis. We won't use these for the script at the next notebook. Therefore, you can ignore the rest.**

In [54]:
df_agg['date_year'] = df_agg['pickup_time'].dt.year
df_agg['date_month'] = df_agg['pickup_time'].dt.month
df_agg['date_day'] = df_agg['pickup_time'].dt.day
df_agg['date_hour'] = df_agg['pickup_time'].dt.hour
df_agg['date_week_of_year'] = df_agg['pickup_time'].dt.isocalendar().week
df_agg['date_quarter'] = df_agg['pickup_time'].dt.quarter


In [55]:
df_agg

,pickup_time,PULocationID,trip_distance,total_amount,duration,count,date_year,date_month,date_day,date_hour,date_week_of_year,date_quarter
0,2019-01-31 22:00:00,7,0.760000,5.800000,2.800000,1.0,2019,1,31,22,5,1
1,2019-01-31 23:00:00,4,4.660000,20.160000,15.450000,1.0,2019,1,31,23,5,1
2,2019-01-31 23:00:00,7,0.650000,5.300000,2.700000,1.0,2019,1,31,23,5,1
3,2019-02-01 00:00:00,4,1.904444,11.807778,9.864815,18.0,2019,2,1,0,5,1
4,2019-02-01 00:00:00,7,1.781667,9.592500,7.538889,12.0,2019,2,1,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2601,2019-02-28 22:00:00,10,11.790000,36.910000,21.250000,2.0,2019,2,28,22,9,1
2602,2019-02-28 23:00:00,4,2.674091,17.392500,13.003030,44.0,2019,2,28,23,9,1
2603,2019-02-28 23:00:00,7,2.104444,11.817778,9.396296,9.0,2019,2,28,23,9,1
2604,2019-02-28 23:00:00,9,5.710000,23.960000,13.016667,3.0,2019,2,28,23,9,1


In [56]:
'''
add lag features
'''
ids = df_agg['PULocationID'].unique()
# may up to 9 hours history is related to the future. May be 24 hours.
for n in range(1, 9):
    c = 'count_lag_' + str(n)
    df_agg[c] = np.NaN
    for i in ids:
        df_agg.loc[df_agg['PULocationID']==i, c] \
            = df_agg[df_agg['PULocationID']==i].sort_values(by=['pickup_time'])['count'].shift(periods=n)
for n in range(1, 9):
    c = 'count_lag_' + str(n)
    df_agg[c] = df_agg[c].fillna(0)

In [57]:
df_agg

,pickup_time,PULocationID,trip_distance,total_amount,duration,count,date_year,date_month,date_day,date_hour,date_week_of_year,date_quarter,count_lag_1,count_lag_2,count_lag_3,count_lag_4,count_lag_5,count_lag_6,count_lag_7,count_lag_8
0,2019-01-31 22:00:00,7,0.760000,5.800000,2.800000,1.0,2019,1,31,22,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-31 23:00:00,4,4.660000,20.160000,15.450000,1.0,2019,1,31,23,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-31 23:00:00,7,0.650000,5.300000,2.700000,1.0,2019,1,31,23,5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-02-01 00:00:00,4,1.904444,11.807778,9.864815,18.0,2019,2,1,0,5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-02-01 00:00:00,7,1.781667,9.592500,7.538889,12.0,2019,2,1,0,5,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601,2019-02-28 22:00:00,10,11.790000,36.910000,21.250000,2.0,2019,2,28,22,9,1,1.0,7.0,1.0,7.0,5.0,14.0,5.0,6.0
2602,2019-02-28 23:00:00,4,2.674091,17.392500,13.003030,44.0,2019,2,28,23,9,1,52.0,34.0,20.0,25.0,21.0,22.0,15.0,16.0
2603,2019-02-28 23:00:00,7,2.104444,11.817778,9.396296,9.0,2019,2,28,23,9,1,16.0,9.0,10.0,12.0,10.0,12.0,8.0,12.0
2604,2019-02-28 23:00:00,9,5.710000,23.960000,13.016667,3.0,2019,2,28,23,9,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [59]:
!pip install tsfresh
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import extract_features
tsfresh_features = extract_features(df_agg[['count', 'PULocationID', 'pickup_time']],
                                    column_id='PULocationID', 
                                    column_sort='pickup_time',
                                    default_fc_parameters=MinimalFCParameters())

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 2.0 MB/s eta 0:00:0000:01


Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 1171.53it/s]


In [60]:
tsfresh_features

,count__sum_values,count__median,count__mean,count__length,count__standard_deviation,count__variance,count__root_mean_square,count__maximum,count__absolute_maximum,count__minimum
1,487.0,1.0,1.777372,274.0,1.112995,1.238758,2.097096,6.0,6.0,1.0
2,11.0,1.0,1.000000,11.0,0.000000,0.000000,1.000000,1.0,1.0,1.0
3,230.0,1.0,1.493506,154.0,0.758046,0.574633,1.674872,5.0,5.0,1.0
4,13491.0,15.0,20.075893,672.0,18.638683,347.400490,27.394196,160.0,160.0,1.0
5,19.0,1.0,1.117647,17.0,0.470588,0.221453,1.212678,3.0,3.0,1.0
6,29.0,1.0,1.074074,27.0,0.261891,0.068587,1.105542,2.0,2.0,1.0
7,10367.0,14.0,15.358519,675.0,7.572390,57.341094,17.123819,58.0,58.0,1.0
8,88.0,1.0,1.142857,77.0,0.417609,0.174397,1.216766,3.0,3.0,1.0
9,169.0,1.0,1.352000,125.0,0.596738,0.356096,1.477836,4.0,4.0,1.0
10,2752.0,4.0,4.794425,574.0,3.309721,10.954255,5.825870,20.0,20.0,1.0


In [61]:
tsfresh_features.columns

Index(['count__sum_values', 'count__median', 'count__mean', 'count__length',
       'count__standard_deviation', 'count__variance',
       'count__root_mean_square', 'count__maximum', 'count__absolute_maximum',
       'count__minimum'],
      dtype='object')

In [62]:
df_agg['PULocationID'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [63]:
'''
add lag features
'''
for c in tsfresh_features.columns:
    df_agg[c] = np.NaN
    
ids = df_agg['PULocationID'].unique()
# may up to 9 hours history is related to the future. May be 24 hours.
for i in ids:
    for c in tsfresh_features.columns:
        df_agg.loc[df_agg['PULocationID']==i, c] = tsfresh_features.loc[i, c]


In [64]:
df_agg

,pickup_time,PULocationID,trip_distance,total_amount,duration,count,date_year,date_month,date_day,date_hour,...,count__sum_values,count__median,count__mean,count__length,count__standard_deviation,count__variance,count__root_mean_square,count__maximum,count__absolute_maximum,count__minimum
0,2019-01-31 22:00:00,7,0.760000,5.800000,2.800000,1.0,2019,1,31,22,...,10367.0,14.0,15.358519,675.0,7.572390,57.341094,17.123819,58.0,58.0,1.0
1,2019-01-31 23:00:00,4,4.660000,20.160000,15.450000,1.0,2019,1,31,23,...,13491.0,15.0,20.075893,672.0,18.638683,347.400490,27.394196,160.0,160.0,1.0
2,2019-01-31 23:00:00,7,0.650000,5.300000,2.700000,1.0,2019,1,31,23,...,10367.0,14.0,15.358519,675.0,7.572390,57.341094,17.123819,58.0,58.0,1.0
3,2019-02-01 00:00:00,4,1.904444,11.807778,9.864815,18.0,2019,2,1,0,...,13491.0,15.0,20.075893,672.0,18.638683,347.400490,27.394196,160.0,160.0,1.0
4,2019-02-01 00:00:00,7,1.781667,9.592500,7.538889,12.0,2019,2,1,0,...,10367.0,14.0,15.358519,675.0,7.572390,57.341094,17.123819,58.0,58.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2601,2019-02-28 22:00:00,10,11.790000,36.910000,21.250000,2.0,2019,2,28,22,...,2752.0,4.0,4.794425,574.0,3.309721,10.954255,5.825870,20.0,20.0,1.0
2602,2019-02-28 23:00:00,4,2.674091,17.392500,13.003030,44.0,2019,2,28,23,...,13491.0,15.0,20.075893,672.0,18.638683,347.400490,27.394196,160.0,160.0,1.0
2603,2019-02-28 23:00:00,7,2.104444,11.817778,9.396296,9.0,2019,2,28,23,...,10367.0,14.0,15.358519,675.0,7.572390,57.341094,17.123819,58.0,58.0,1.0
2604,2019-02-28 23:00:00,9,5.710000,23.960000,13.016667,3.0,2019,2,28,23,...,169.0,1.0,1.352000,125.0,0.596738,0.356096,1.477836,4.0,4.0,1.0
